In [1]:
import pandas as pd
import statistics

In [2]:
data = pd.read_csv('./measurements.csv')
data.head()

,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun,refill liters,refill gas
0,28,5,26,"21,5",12,NaN,E10,0,0,0,45,E10
1,12,"4,2",30,"21,5",13,NaN,E10,0,0,0,NaN,NaN
2,"11,2","5,5",38,"21,5",15,NaN,E10,0,0,0,NaN,NaN
3,"12,9","3,9",36,"21,5",14,NaN,E10,0,0,0,NaN,NaN
4,"18,5","4,5",46,"21,5",15,NaN,E10,0,0,0,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   distance       388 non-null    object
 1   consume        388 non-null    object
 2   speed          388 non-null    int64 
 3   temp_inside    376 non-null    object
 4   temp_outside   388 non-null    int64 
 5   specials       93 non-null     object
 6   gas_type       388 non-null    object
 7   AC             388 non-null    int64 
 8   rain           388 non-null    int64 
 9   sun            388 non-null    int64 
 10  refill liters  13 non-null     object
 11  refill gas     13 non-null     object
dtypes: int64(5), object(7)
memory usage: 36.5+ KB


In [4]:
# CORREGIMOS DISTANCE
lista_corregida = []
for e in data.distance:
    if type(e) == str:
        lista_corregida.append(float(e.replace(',','.')))
    else:
        lista_corregida.append(max(lista_corregida))
data.distance = lista_corregida

In [5]:
# CORREGIMOS CONSUME
lista_corregida = []
for e in data.consume:
    if type(e) == str:
        lista_corregida.append(float(e.replace(',','.')))
    else:
        lista_corregida.append(max(lista_corregida))
data.consume = lista_corregida

In [6]:
# CORREGIMOS TEMP_INSIDE
lista_corregida = []
for e in data.temp_inside:
    if type(e) == str:
        lista_corregida.append(float(e.replace(',','.')))
    else:
        lista_corregida.append(max(lista_corregida))
data.temp_inside = lista_corregida

In [7]:
data.specials.value_counts() # Muchos nulos, me la cargo

rain                  32
sun                   27
AC rain                9
ac                     8
AC                     6
snow                   3
sun ac                 3
AC snow                1
half rain half sun     1
AC sun                 1
AC Sun                 1
ac rain                1
Name: specials, dtype: int64

In [8]:
data.drop('specials', axis=1, inplace=True)

In [9]:
data.gas_type.value_counts() # Está OK

SP98    228
E10     160
Name: gas_type, dtype: int64

In [10]:
data.AC.value_counts() # Está OK, pero dudo de la utilidad

0    358
1     30
Name: AC, dtype: int64

In [11]:
data.rain.value_counts() # Está OK, pero dudo de la utilidad

0    340
1     48
Name: rain, dtype: int64

In [12]:
data.sun.value_counts() # Está OK, pero dudo de la utilidad

0    356
1     32
Name: sun, dtype: int64

In [13]:
data['refill liters'].value_counts() # Muchos nulos, no tiene utilidad

45      2
37,7    2
39      2
37,6    1
38      1
38,3    1
10      1
41      1
37      1
37,2    1
Name: refill liters, dtype: int64

In [14]:
data['refill gas'].value_counts() # Muchos nulos, no tiene utilidad

SP98    8
E10     5
Name: refill gas, dtype: int64

In [15]:
data.drop(['refill liters', 'refill gas'], axis=1, inplace=True)

In [16]:
data.to_excel('./measurements_view.xlsx')

In [17]:
# Hacemos una regla de 3... Si consume 5 a los 100, ¿cuantos consume a los 28km? 5*28/100

data['litros_consumidos'] = (data.consume*data.distance)/100

In [18]:
data

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,litros_consumidos
0,28.0,5.0,26,21.5,12,E10,0,0,0,1.4000
1,12.0,4.2,30,21.5,13,E10,0,0,0,0.5040
2,11.2,5.5,38,21.5,15,E10,0,0,0,0.6160
3,12.9,3.9,36,21.5,14,E10,0,0,0,0.5031
4,18.5,4.5,46,21.5,15,E10,0,0,0,0.8325
...,...,...,...,...,...,...,...,...,...,...
383,16.0,3.7,39,24.5,18,SP98,0,0,0,0.5920
384,16.1,4.3,38,25.0,31,SP98,1,0,0,0.6923
385,16.0,3.8,45,25.0,19,SP98,0,0,0,0.6080
386,15.4,4.6,42,25.0,31,SP98,1,0,0,0.7084


In [19]:
# Añadimos los precios: E10 = 1.38, SP98 = 1.46
precios = {"E10":1.38, "SP98":1.46}

coste_viaje = []
for i,gas in enumerate(data.gas_type):
    if gas == "E10":
        coste_viaje.append(1.38*(data['litros_consumidos'][i]))
    else:
        coste_viaje.append(1.46*(data['litros_consumidos'][i]))
data['coste_viaje'] = coste_viaje

In [20]:
data

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,litros_consumidos,coste_viaje
0,28.0,5.0,26,21.5,12,E10,0,0,0,1.4000,1.932000
1,12.0,4.2,30,21.5,13,E10,0,0,0,0.5040,0.695520
2,11.2,5.5,38,21.5,15,E10,0,0,0,0.6160,0.850080
3,12.9,3.9,36,21.5,14,E10,0,0,0,0.5031,0.694278
4,18.5,4.5,46,21.5,15,E10,0,0,0,0.8325,1.148850
...,...,...,...,...,...,...,...,...,...,...,...
383,16.0,3.7,39,24.5,18,SP98,0,0,0,0.5920,0.864320
384,16.1,4.3,38,25.0,31,SP98,1,0,0,0.6923,1.010758
385,16.0,3.8,45,25.0,19,SP98,0,0,0,0.6080,0.887680
386,15.4,4.6,42,25.0,31,SP98,1,0,0,0.7084,1.034264


In [21]:
eficiencia = []
for i,e in enumerate(data['distance']):
    eficiencia.append(e/(data['litros_consumidos'][i]))
data['eficiencia'] = eficiencia

In [22]:
data.groupby('gas_type').mean()[['coste_viaje', 'eficiencia']]

,coste_viaje,eficiencia
gas_type,,
E10,1.392327,20.814378
SP98,1.290083,21.163755


In [23]:
# Es más eficience el SP98 tanto a nivel de km por litros como a nivel económico